In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1234)

In [2]:
cols = ['f1', 'f2', 'f3', 'f4', 'species']
data = pd.read_csv('data/iris.csv', names=cols)

In [3]:
data.head()

,f1,f2,f3,f4,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,5.0,3.4,1.5,0.2,Iris-setosa
3,4.4,2.9,1.4,0.2,Iris-setosa
4,4.9,3.1,1.5,0.1,Iris-setosa


In [4]:
X = data[cols[0:-1]]
X = MinMaxScaler().fit_transform(X)
y = pd.get_dummies(data['species'])

In [5]:
X[0:5]

array([[ 0.22222222,  0.625     ,  0.06779661,  0.04166667],
       [ 0.16666667,  0.41666667,  0.06779661,  0.04166667],
       [ 0.19444444,  0.58333333,  0.08474576,  0.04166667],
       [ 0.02777778,  0.375     ,  0.06779661,  0.04166667],
       [ 0.16666667,  0.45833333,  0.08474576,  0.        ]])

In [6]:
y.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [7]:
input_size = 4
num_classes = 3
hidden_size = 5
learning_rate = 0.1
num_epoch = 1000

#build model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
net = Net(input_size, hidden_size, num_classes)

#choose optimizer and loss function
criterion = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

#train
for epoch in range(num_epoch):
    X_tensor = Variable(torch.from_numpy(X).float())
    Y_tensor = Variable(torch.from_numpy(y.as_matrix()).float())

    #feedforward - backprop
    optimizer.zero_grad()
    out = net(X_tensor)
    loss = criterion(out, Y_tensor)
    loss.backward()
    optimizer.step()

In [8]:
test_infer = pd.read_csv('data/test.csv', names=cols[0:-1])
test_infer = MinMaxScaler().fit_transform(test_infer)

In [9]:
X_test = Variable(torch.from_numpy(test_infer).float())
out = net(X_test)
_, labels = torch.max(out.data, 1)

In [10]:
species = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
predictions = []
for label in labels:
    predictions.append(species[label])

In [11]:
predictions

['Iris-versicolor',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-setosa']

In [13]:
type(y.as_matrix())

numpy.ndarray